#


In [ ]:
!pip install crewai


In [ ]:
!pip install crewai_tools


In [ ]:
!pip install python-dotenv


In [ ]:
!pip install docling

In [ ]:
from crewai import Agent, Task, Crew, LLM
# from langchain_groq import ChatGroq

In [ ]:
from google.colab import userdata
import os
# os.environ["GROQ_API_KEY"] = userdata.get("MY_GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")


In [ ]:
import os
from crewai import Agent, Task, Crew, LLM

from dotenv import load_dotenv
load_dotenv("/content/keys.env.txt")

llm = LLM(model="groq/qwen/qwen3-32b", temperature=0.7)
# llm = LLM(model="groq/gemma2-9b-it", temperature=0.7)
#------------------------------------------------------------------------

from typing import Type
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from google import generativeai as genai
from google.generativeai import types
#-------------------------------------------------------------------
class GeminiSearchInput(BaseModel):
    query: str = Field(..., description="Give me information on {topic}")

# -------------------------------------------------
# Gemini Search Tool
class GeminiSearchTool(BaseTool):
    name: str = "Gemini Web Search"
    description: str = "Performs real-time web search using Google Gemini with grounding."
    args_schema: Type[BaseModel] = GeminiSearchInput

    def _run(self, query: str) -> str:
        client = genai.Client()  # GEMINI_API_KEY must be set in env
        grounding_tool = types.Tool(google_search=types.GoogleSearch())
        config = types.GenerateContentConfig(tools=[grounding_tool], temperature=0.0)

        response = client.models.generate_content(
            model="gemini-2.5-flash",  # or "gemini-2.5-pro"
            contents=query,
            config=config,
        )
        return response.text
#-------------------------------------------------------------------
# Create agents with your tool
researcher = Agent(
    role="Researcher",
    goal="""Search the web for information on {topic}. Find accurate information.""",
    backstory="""You are an experienced web researcher to find useful information on {topic}. Get values from the web for more accurate information.""",
    llm=llm,
    tools=[GeminiSearchTool()],
    verbose=True
)

analysis = Agent(
    role="Analysis",
    goal="You are to take the raw content from the reseracher agent and find the key insights, research findings and important points. Include values from the researcher agents for mor accurate information.",
    backstory="An expert and seasoned analysis who can extract the needed points. ",
    llm=llm,
    verbose=True
)

writer = Agent(
    role="Report Writer",
    goal="""You are to get the content from the analysis agent and write it down in the form of a report.
    The report must be written strictly in valid Markdown format, with the following sections:
    # Topic
    # Key Findings
    # Conclusion
    # References
    Ensure proper Markdown formatting with headings, bullet points for lists, and numbered references.""",
    backstory="You are a seasoned report writer who specializes in creating well-structured Markdown reports.",
    llm=llm,
    verbose=True
)

task1 = Task(
    description="You are to search the web for information on the topic of 'Cancer'. ",
    expected_output="You need to give all the information you collected.",
    agent=researcher,
    verbose=True
)

task2 = Task(
    description="You are to read the information from analysis agent and collect the important insights, key points and topics from it.",
    expected_output="Give the important insights, key points and topics from the information.",
    agent=analysis,
    verbose=True
)

task3 = Task(
    description="Get the information from the analysis agent and write a Markdown report with sections: Topic, Key Findings, Conclusion, References.",
    expected_output="""A well-formatted Markdown report, for example:

# Topic
Cancer

# Key Findings
- Finding 1
- Finding 2

# Conclusion
Summarized conclusion text.

# References
1. Reference 1
2. Reference 2
""",
    agent=writer,
    verbose=True
)

In [ ]:
# Create the Crew
crew = Crew(
    agents=[researcher, analysis, writer],
    tasks=[task1, task2, task3],
    verbose=True
)

In [ ]:
crew.kickoff(inputs={"topic": "Cancer"})